In [1]:
import operator
import os
import sqlite3
from typing import Annotated, Sequence, TypedDict
from langgraph.graph import StateGraph, START, END, MessagesState
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langgraph.types import RetryPolicy

from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
BASE_URL = os.getenv("BASE_URL")

db = SQLDatabase.from_uri("sqlite:///:memory:")

# 初始化 Artists 表
# 创建表
db.run("""
CREATE TABLE Artists (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    genre TEXT,
    country TEXT
);
""")

# 插入数据
db.run("""
INSERT INTO Artists (name, genre, country) VALUES
('The Beatles', 'Rock', 'UK'),
('Taylor Swift', 'Pop', 'USA'),
('Jay Chou', 'Mandopop', 'China'),
('Adele', 'Soul', 'UK'),
('BTS', 'K-pop', 'South Korea');
""")

model = ChatOpenAI(
    model="Qwen/Qwen2.5-7B-Instruct",
    api_key=API_KEY,
    base_url=BASE_URL
)

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

def query_database(state):
    query_result = db.run("SELECT * FROM Artists;")
    return {"messages": [AIMessage(content=query_result)]}

def call_model(state):
    response = model.invoke(state['messages'])
    return {"messages": [response]}

def explain_result(state):
    prompt = f"以下是数据库查询结果，请用自然语言总结它：\n{state['messages'][-1].content}"
    response = model.invoke([HumanMessage(content=prompt)])
    return {"messages": [response]}

# 构建图
builder = StateGraph(AgentState)
builder.add_node("query_database", query_database, retry=RetryPolicy(retry_on=sqlite3.OperationalError))
builder.add_node("model", call_model, retry=RetryPolicy(max_attempts=5))
builder.add_node("explain_result", explain_result)
builder.add_edge(START, "model")
builder.add_edge("model", "query_database")
builder.add_edge("query_database", "explain_result")
builder.add_edge("explain_result", END)
graph = builder.compile()
result = graph.invoke({"messages": [HumanMessage(content="总共有多少位艺术家？")]})
print(result["messages"][-1].content)

这个数据库查询结果包含了5位著名艺人的相关信息，他们分别来自不同的音乐流派和地区。具体来说：

1. The Beatles 是一位来自英国的摇滚乐艺人。
2. Taylor Swift 是一位来自美国的流行乐艺人。
3. Jay Chou 是一位来自中国的华语流行音乐艺人，也被称为国语流行（Mandopop）。
4. Adele 是一位来自英国的灵魂乐艺人。
5. BTS 是一位来自韩国的K-pop音乐团体。
